In [2]:
import pandas as pd
import numpy as np

In [3]:
result = pd.read_csv('../../result/CF_suggest.csv', sep=',', encoding='latin-1')
result.head()

,movie index,user indexes
0,0,39|79|83|91|97|103|120|125|131|167|168|172|186...
1,1,16|20|25|29|74|103|108|131|179|194|197|214|244...
2,2,2|39|58|63|64|79|97|103|110|114|120|121|125|13...
3,3,6|19|22|24|25|35|42|54|55|69|70|74|76|77|92|10...
4,4,NaN


In [4]:
ratings = pd.read_csv('../../data/datasets/rating/ml_detail.csv', sep=',', encoding='latin-1')
ratings.head()

,movie id,user id,rating,content,date,movie index,user index
0,tt0102813,ur0391152,2,I've always had kind of a perverse curiosity a...,22 August 2010,6914,81
1,tt0387808,ur0391152,5,20th Century Fox was moronic not to release th...,6 November 2006,3393,81
2,tt0845046,ur0391152,5,It's the kind of movie that reminds me that th...,28 September 2008,3507,81
3,tt0113824,ur0391152,5,"A very moving animÃ© film from Studio Ghibli, ...",29 September 2003,3181,81
4,tt0455538,ur0391152,2,Based on the autobiographical tale of writer T...,6 October 2008,4699,81


In [10]:
result.dropna(subset=['user indexes'], inplace=True)

In [ ]:
user_ids = []
for user_group in result['user indexes']:
    if (user_group != np.nan):
        index_list = user_group.split('|')
        id_list = []
        for index in index_list:
            index = int(index)
            if (len(ratings[ratings['user index'] == index])):
                id_list.append(ratings[ratings['user index'] == index]['user id'].tolist()[0])
                
        user_ids.append('|'.join(id_list))
user_ids

In [28]:
movie_ids = []
for movie_index in result['movie index']:
    if len(ratings[ratings['movie index'] == movie_index]):
        movie_ids.append(ratings[ratings['movie index'] == movie_index]['movie id'].tolist()[0])

In [29]:
s_dict = dict()
for i in range(len(user_ids)):
    user_id_list = user_ids[i].split("|")
    movie_id = movie_ids[i]
    for user_id in user_id_list:
        if user_id in s_dict:
            s_dict.update({user_id: "|".join(s_dict[user_id].split("|") + [movie_id])})
        else:
            s_dict.update({user_id: movie_id})

In [30]:
re = pd.DataFrame(columns=['user id', 'movie id'])
re['user id'] = list(s_dict.keys())
re['movie id'] = list(s_dict.values())

In [38]:
re.to_csv('../../result/CF/CF_suggest_user_movies.csv', sep=',', encoding='latin-1', index=False)

In [31]:
history = pd.DataFrame(columns=['user id', 'movie id'])

In [32]:
hs = []
for user_id in re['user id']:
    watched_movies = "|".join(ratings[ratings['user id'] == user_id]['movie id'].tolist())
    hs.append(watched_movies)
history['user id'] = re['user id']
history['movie id'] = hs

In [37]:
history.to_csv('../../result/CF/CF_history_user_movies.csv', sep=',', encoding='latin-1', index=False)